In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import plotly.express as px
#import seaborn as sns
import requests
from io import StringIO

In [3]:
# Entire_home_apt.csv
# id_neighbordhood_date.csv

#dwn_url='https://figshare.com/articles/dataset/Entire_home_apt_csv/13259795'
#url = requests.get(dwn_url).text
#csv_raw = StringIO(url)

#url

In [21]:
df=pd.read_csv('../Final/Berlin/merge_data_files/Entire_home_apt.csv',low_memory=False,index_col=0) 
df

,id,latitude,longitude,listing_url,host_id,host_name,host_since,room_type,neighbourhood_cleansed,neighbourhood_group_cleansed,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location.1,review_scores_value,calculated_host_listings_count,date
0,2602460,52.60010,13.32972,https://www.airbnb.com/rooms/2602460,13315290,Flora,2014-03-19,Entire home/apt,MV 2,Reinickendorf,...,2015-09-15,93.0,9.0,10.0,10.0,10.0,9.0,9.0,1,2015-10-03
1,3637595,52.49110,13.41359,https://www.airbnb.com/rooms/3637595,4094666,Patricia Aulitzky,2012-11-08,Entire home/apt,Tempelhofer Vorstadt,Friedrichshain-Kreuzberg,...,2015-09-30,98.0,10.0,10.0,10.0,10.0,10.0,9.0,1,2015-10-03
2,6160418,52.49603,13.37990,https://www.airbnb.com/rooms/6160418,1718617,Guy,2012-02-08,Entire home/apt,Tempelhofer Vorstadt,Friedrichshain-Kreuzberg,...,2015-05-05,100.0,10.0,10.0,10.0,10.0,8.0,8.0,2,2015-10-03
3,3588362,52.49348,13.39693,https://www.airbnb.com/rooms/3588362,16739581,Ava,2014-06-13,Entire home/apt,Tempelhofer Vorstadt,Friedrichshain-Kreuzberg,...,2015-09-14,87.0,9.0,10.0,9.0,8.0,9.0,8.0,2,2015-10-03
4,6655897,52.48871,13.41666,https://www.airbnb.com/rooms/6655897,34836886,Anna,2015-06-02,Entire home/apt,Tempelhofer Vorstadt,Friedrichshain-Kreuzberg,...,2015-07-18,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1,2015-10-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005459,31382592,52.51236,13.40905,https://www.airbnb.com/rooms/31382592,235031879,Helga,2019-01-07,Entire home/apt,Alexanderplatz,Mitte,...,2020-08-26,98.0,10.0,10.0,10.0,10.0,10.0,10.0,1,2020-08-30
1005460,31382644,52.46875,13.42709,https://www.airbnb.com/rooms/31382644,235032226,Benjamin,2019-01-07,Entire home/apt,Schillerpromenade,Neukölln,...,2020-08-20,98.0,10.0,10.0,10.0,10.0,10.0,10.0,1,2020-08-30
1005462,31385396,52.50923,13.46516,https://www.airbnb.com/rooms/31385396,235058174,Bert,2019-01-07,Entire home/apt,Frankfurter Allee Süd FK,Friedrichshain-Kreuzberg,...,2020-08-21,94.0,10.0,10.0,10.0,10.0,10.0,9.0,1,2020-08-30
1005463,31386775,52.52315,13.38748,https://www.airbnb.com/rooms/31386775,1581930,Darius,2012-01-08,Entire home/apt,Alexanderplatz,Mitte,...,2020-06-04,97.0,10.0,10.0,9.0,9.0,10.0,9.0,2,2020-08-30


In [22]:
df.date


0          2015-10-03
1          2015-10-03
2          2015-10-03
3          2015-10-03
4          2015-10-03
              ...    
1005459    2020-08-30
1005460    2020-08-30
1005462    2020-08-30
1005463    2020-08-30
1005465    2020-08-30
Name: date, Length: 514807, dtype: object

In [9]:
max(df.date)

Timestamp('2020-08-30 00:00:00')

In [10]:
import pydeck as pdk
import pandas as pd

CPU_GRID_LAYER_DATA = (
    "https://raw.githubusercontent.com/uber-common/" "deck.gl-data/master/website/sf-bike-parking.json"
)
df = pd.read_json(CPU_GRID_LAYER_DATA)

# Define a layer to display on a map

layer = pdk.Layer(
    "GridLayer",
    df,
    pickable=True,
    extruded=True,
    cell_size=200,
    elevation_scale=4,
    get_position="COORDINATES",
)

view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=11, bearing=0, pitch=45)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "{position}\nCount: {count}"},
)
r.to_html("grid_layer.html")

AttributeError: module 'pydeck' has no attribute 'Layer'

In [ ]:

df.loc[df.date=='2020-05-14'].host_id.nunique()

In [ ]:
#find the top 20 districts

top10=df.loc[df.date=='2020-05-14']['neighbourhood_cleansed'].value_counts()[:10].index.tolist()
top10

In [ ]:
df2=df.groupby(['neighbourhood_cleansed','date'],as_index=False).count()[['neighbourhood_cleansed','date','id']]

In [ ]:
#df2=df2.sort_values(by=['id'], ascending=False)
df2.date = pd.to_datetime(df2.date, format='%Y-%m-%d')

In [ ]:
toplot=df2.loc[df2['neighbourhood_cleansed'].isin(top15)]
toplot

In [ ]:
tp=toplot.pivot(index='date', columns='neighbourhood_cleansed', values='id').reset_index().rename_axis(None, axis=1)

In [ ]:



import plotly.express as px
toplot = df2.loc[df2['neighbourhood_cleansed'].isin(top10)]
fig = px.line(toplot, x="date", y="id", color='neighbourhood_cleansed')
fig.show()

In [ ]:
o = px.data.gapminder().query("continent=='Oceania'")
o

In [ ]:
#find the top 20 hots

top20=df.loc[df.date=='2020-05-14']['host_id'].value_counts()[:30].index.tolist()
top20

In [ ]:
df3=df.groupby(['host_id','date'],as_index=False).count()[['host_id','date','id']]

In [ ]:
df3=df3.sort_values(by=['id'], ascending=False)
df3.date = pd.to_datetime(df3.date, format='%Y-%m-%d')

In [ ]:
toplot=df3.loc[df3['host_id'].isin(top20)]

In [ ]:
toplot

In [ ]:
toplot=df3.loc[df3['host_id'].isin(top20)]

plt.figure(figsize=(15, 10)) 



sns.lineplot(data=toplot, x="date", y="id", hue='host_id', markers=True, dashes=False)

In [ ]:
toplot=df3.copy()

bins = [0,1,2,4,8,16,260]

toplot['binned'] = pd.cut(toplot['id'], bins)



toplot

In [ ]:
df['binned_calc']=pd.cut(df['calculated_host_listings_count'], bins)

In [ ]:
df['binned_calc']=df['binned_calc'].astype(str)

In [ ]:
pie_5=df.groupby(['binned_calc','date'],as_index=False).count()[['binned_calc','date','id']]

In [ ]:
pie_5.date

In [ ]:
pie_5.loc[pie_5.date=='2016-05-03']

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots



fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['2016','2020'])
fig.add_trace(go.Pie(labels=pie_5.loc[pie_5.date=='2016-05-03'].binned_calc, 
                     values=pie_5.loc[pie_5.date=='2016-05-03'].id, scalegroup='one',
                     name="2016"), 1, 1)
fig.add_trace(go.Pie(labels=pie_5.loc[pie_5.date=='2020-05-14'].binned_calc, 
                     values=pie_5.loc[pie_5.date=='2020-05-14'].id, scalegroup='one',
                     name="2020"), 1, 2)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0), height=600
)

fig.update_layout(title_text='Multiple flat host evolution')
fig.show()
fig.write_html("../Final/Pie_chart_5years.html")

In [ ]:
# count the multiple host listings
plot=toplot.groupby(['binned','date'],as_index=False).count()

In [ ]:
fig = px.bar(plot, x='date', y='id', labels = {"value": "count", "variable": "medal"},
             color='binned', 
            title='Number of hosts with multiple listings_only_apts')
fig.show()
fig.write_html("../Final/multipleL_host_5years.html")

# plot the multiple flat hosts over time on map

In [ ]:
#toplot.loc[
toplot['binned']=toplot['binned'].astype(str)


In [ ]:
data2=toplot.loc[toplot['binned'].isin(['(2, 4]','(4, 8]','(8, 16]','(16, 260]'])][['date','host_id','binned']]

In [ ]:
df.dtypes

In [ ]:
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')

In [ ]:
data2.dtypes

In [ ]:
data=pd.merge(df, data2,  how='left',  on=['host_id','date'])


In [ ]:
# clean data for ploting
#df=data.loc[data.binned.notnull()]
df.date=df.date.apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
import plotly.figure_factory as ff

# it needs a token for access mapbox
px.set_mapbox_access_token('pk.eyJ1Ijoiam9obnppbnoiLCJhIjoiY2tmbWthazZ6MDNueDJxb2ZyZ2M3czU0dyJ9.Bl3T4kl14xan7glGxid_Rw')


lat = df['latitude']
lon = df['longitude']
frame = df['date']

fig = ff.create_hexbin_mapbox(
    lat=lat, lon=lon, nx_hexagon=50, animation_frame=frame,
    color_continuous_scale="Viridis", labels={"color": "Airbnb listings > 50", "frame": "Period"},
    opacity=0.5, min_count=60, height=500, zoom=10)
    #show_original_data=True, original_data_marker=dict(opacity=0.4, size=3, color="blue"))
fig.update_layout(title={'text': "density spots"})
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()

# write to an html file
fig.write_html("../Final/hives_for_density2.html")